In [1]:
%%capture
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
!pip install openpyxl
!pip install qpsolvers
import qpsolvers
!pip install qpsolvers[cvxopt]
!pip install qpsolvers[open_source_solvers]
!pip install qpsolvers[clarabel]
from qpsolvers import solve_qp

In [2]:
def rbf(x,y,l=1):
    """Gaussian kernel

    Parameters
    -------------------------------
    x : float
    a real number

    y : float
    a real number

    l: float, non zero
    a scale parameter
    -------------------------------
    """
    dim = x.shape[0]
    vect = np.empty(dim)
    if dim == y.shape[0]  :
        d = np.exp((-1)*((np.linalg.norm(x-y))/(2*(l**2))))
        return d
    else :
        for i in range(dim):
            vect[i] = np.exp((-1)*(np.linalg.norm(x[i] - y))/(2*(l**2)))
        return vect

In [3]:
max_calculated = pd.read_csv('max_calculated.csv')
max_elemental = pd.read_csv('max_elemental.csv')
list_mxene = pd.read_excel('synthesized-MXenes-MAX.xlsx',sheet_name=0)
list_failed = pd.read_excel('synthesized-MXenes-MAX.xlsx', sheet_name=2)
n_samples = max_elemental.shape[0]

In [4]:
synth_list = pd.unique(list_mxene['MXene'])[:-1]
to_drop = list(range(167,173))
mx_ene_df = list_mxene.drop(labels = to_drop, axis='index')
mx_ene_df = mx_ene_df.drop(['Unnamed: 9','Unnamed: 12','Notes','status','Reference method'],axis=1)
max_elemental['class'] = np.zeros(max_elemental.shape[0])
parents = mx_ene_df['Parent material'].unique()
banned_words = ['+','Mxene','topochemical','reaction', 'or',
               'synthesis','MXene','direct']
complete_parents = []
for i in range(len(parents)):
    inter = parents[i].split()
    for word in range(len(inter)):
        if inter[word] not in banned_words:
            complete_parents.append(inter[word])


for i in range(max_elemental.shape[0]):
    if max_elemental.loc[i,'compound_formula'] in complete_parents:
        max_elemental.loc[i,'class'] = 1

max_elemental = max_elemental.set_index('compound_formula',drop=True)
max_elemental = max_elemental.drop(['M_element', 'A_element', 'X_element'],axis=1)
test_tree = DecisionTreeClassifier().fit(X=max_elemental.drop(['class'],axis=1),
                                          y=max_elemental['class'])

imp_feat = test_tree.feature_importances_
names_feat = test_tree.feature_names_in_
df_imp_feat = pd.DataFrame(np.hstack((imp_feat.reshape(imp_feat.shape[0],1),names_feat.reshape(imp_feat.shape[0],1))))
df_imp_feat.columns = ['features', 'name']
df_imp_feat = df_imp_feat.sort_values('features', ascending=False)

df_diff_z = df_imp_feat[df_imp_feat['features'] != 0]


failed = list_failed['MAX']
failed = list(failed)

for i in max_elemental.index:
    if i in failed:
        max_elemental.loc[i,'class'] = -1


number_of_atoms = np.empty(n_samples)
compteur = 0
for element in max_elemental.index:
    inter = []
    for cara in element:
        if cara in list(str(1234567890)):
            inter.append(cara)
    if len(inter) == 1:
        number_of_atoms[compteur] = int(inter[0]) + 2
    elif len(inter) == 2:
        number_of_atoms[compteur] = int(inter[0]) + int(inter[1]) + 1
    elif len(inter) == 3:
        number_of_atoms[compteur] = int(inter[0]) + int(inter[1]) + int(inter[2])
    compteur += 1


columns_name = max_elemental.columns.copy()
normalized = max_elemental.drop(['class'],axis=1).to_numpy()/number_of_atoms.reshape(n_samples,1)
max_elem_norm = pd.DataFrame(normalized)
max_elem_norm['class'] = max_elemental['class'].copy()
max_elem_norm.columns = columns_name
max_elem_norm['compound_name'] = max_elemental.index
max_elem_norm = max_elem_norm.set_index('compound_name',drop=True)

max_elem_norm['class'] = max_elemental['class'].copy()
list_of_imp_names = list(df_diff_z['name'])
list_of_imp_names.append('label')
list_of_imp_names.append('class')
max_elem_norm = max_elem_norm.filter(items=list_of_imp_names, axis=1)
max_elem_norm['label'] = np.zeros(n_samples)
for i in max_elem_norm.index:
    if max_elem_norm.loc[i,'class'] == 1:
        max_elem_norm.loc[i,'label'] = 1
    else:
        max_elem_norm.loc[i,'label'] = -1

positive_samples = max_elem_norm[max_elem_norm['label'] == 1]
unlabelled_samples = max_elem_norm[max_elem_norm['label'] == -1]



In [5]:
true_max_elem = max_elem_norm.copy()
list_of_positive_names = positive_samples.index
result_for_positive_instances = np.zeros(positive_samples.shape[0])
gen_count = 0

for particular_element in list_of_positive_names:
    element_to_test = true_max_elem.drop(['class','label'],axis=1).loc[particular_element,:].to_numpy()
    max_elem_norm = true_max_elem.drop(particular_element,axis=0)

    
    svm_train = SVC(kernel='sigmoid', probability = True).fit(X=max_elem_norm.to_numpy()[:,:-2],
                                                          y=max_elem_norm.to_numpy()[:,-1])
    probas = svm_train.predict_proba(max_elem_norm.to_numpy()[:,:-2])
    proba_gap = np.zeros(n_samples -1)
    for i in range(n_samples -1):
        proba_gap[i] = probas[i,1] - probas[i,0]


    max_elem_norm['proba_gap'] = proba_gap

    n_min = 4 #we increase it a bit as the classes are really unbalanced

    l_boundary = np.mean(np.sort(max_elem_norm[max_elem_norm['label'] == 1]['proba_gap'])[:n_min])


    relab = np.zeros(n_samples -1)
    for i in range(n_samples -1):
        if max_elem_norm.loc[max_elem_norm.index[i],'proba_gap'] < l_boundary:
            relab[i] = -1
        elif max_elem_norm.loc[max_elem_norm.index[i],'label'] == 1 or max_elem_norm.loc[max_elem_norm.index[i],'proba_gap'] >= 0:
            relab[i] = 1
        else:
            relab[i] = 0
    max_elem_norm['relab'] = relab

    for i in range(n_samples -1):
        if max_elem_norm.loc[max_elem_norm.index[i],'label'] == 1:
            max_elem_norm.loc[max_elem_norm.index[i],'relab'] = 1
    B=10000
    labeled_data = max_elem_norm[max_elem_norm['relab'] != 0].copy()
    output_labeled = labeled_data['relab'].to_numpy()
    list_of_index = labeled_data.index
    labeled_data = labeled_data.reset_index(drop=True)
    labeled_data = labeled_data.to_numpy()[:,:-4]
    unlabeled_data = max_elem_norm.drop(index=list_of_index,axis=0)
    unlabeled_data = unlabeled_data.to_numpy()[:,:-4]
    n_unlabeled = unlabeled_data.shape[0]
    n_labels = labeled_data.shape[0]
    capital_k = np.zeros((n_labels,n_labels))
    kappa = np.zeros(n_labels)


    #construction of capital_k
    for i in range(n_labels):
        for j in range(i,n_labels):
            capital_k[i,j] = rbf(x=labeled_data[i,:],y=labeled_data[j,:])

    capital_k = capital_k + capital_k.T
    for i in range(n_labels):
        capital_k[i,i] = 1

    capital_k[np.where(np.isnan(capital_k) == True)] = 0

    #construction of kappa
    ratio_lab_unlab = n_labels/n_unlabeled

    for i in range(n_labels):
        vector = np.zeros(n_unlabeled)
        for k in range(n_unlabeled):
            vector[k] = rbf(x=labeled_data[i,:],y=unlabeled_data[k,:])    
        kappa[i] = ratio_lab_unlab*np.sum(vector)

    kappa = -kappa



    ones_transposed = np.ones(n_labels).reshape(1,n_labels)
    a_mat = np.vstack((ones_transposed,ones_transposed*-1,
                   np.eye(n_labels),np.eye(n_labels)*-1))
    epsilon = (np.sqrt(n_labels)-1)/np.sqrt(n_labels)
    ub_mat = np.vstack((n_labels*(1+epsilon),n_labels*(epsilon-1),
                        np.ones(n_labels).reshape(n_labels,1)*B,
                        np.zeros(n_labels).reshape(n_labels,1)))



    beta_opti = solve_qp(P=capital_k,q=kappa,G=a_mat,h=ub_mat,solver='cvxopt')


    svm_weighted = SVC().fit(X=labeled_data,y=output_labeled,sample_weight=beta_opti)

    predictions_weighted = svm_weighted.predict(max_elem_norm.to_numpy()[:,:-4])

    positive = 0
    true_positive = 0
    for i in range(n_samples-1):
        if predictions_weighted[i] == 1:
            positive += 1
            if max_elem_norm.loc[max_elem_norm.index[i],'class'] == 1:
                true_positive += 1


    recall_pgpu = true_positive/max_elem_norm[max_elem_norm['class'] == 1].shape[0]
    weird_estim_pgpu = (recall_pgpu**2)/(positive/max_elem_norm.shape[0])
    print('the element taken out of the sample is : ', particular_element, '\n',
        'the recall of pgpu on train set is :', recall_pgpu, '\n', 
      'the weird metric of pgpu on train set is :', weird_estim_pgpu)
    element_to_test = element_to_test.reshape(1,-1)
    result_for_positive_instances[gen_count] = svm_weighted.predict(element_to_test)[0]


    


the element taken out of the sample is :  Ti2AlC 
 the recall of pgpu on train set is : 0.7857142857142857 
 the weird metric of pgpu on train set is : 1.436030276308054
the element taken out of the sample is :  Ti2SC 
 the recall of pgpu on train set is : 0.7857142857142857 
 the weird metric of pgpu on train set is : 1.4554967972590496
the element taken out of the sample is :  Ti2ZnC 
 the recall of pgpu on train set is : 0.8571428571428571 
 the weird metric of pgpu on train set is : 2.0037911425124935
the element taken out of the sample is :  V2AlC 
 the recall of pgpu on train set is : 0.7857142857142857 
 the weird metric of pgpu on train set is : 1.5024988466861446
the element taken out of the sample is :  Nb2AlC 
 the recall of pgpu on train set is : 0.7857142857142857 
 the weird metric of pgpu on train set is : 1.5008832565284176
the element taken out of the sample is :  Ta2AlC 
 the recall of pgpu on train set is : 1.0 
 the weird metric of pgpu on train set is : 1.489459815